In [11]:
# lib
import matplotlib.pyplot as plt
import torch

In [12]:
# def

# config

num_positions = 500
d_model = 512

def cosine_similarity(vec1, vec2):
    vec1_norm = torch.linalg.norm(vec1) + 0.00000000001
    vec2_norm = torch.linalg.norm(vec2) + 0.00000000001
    return torch.dot(vec1, vec2) / (vec1_norm * vec2_norm)
def cosine_similarity_matrix(vectors):
    return torch.tensor([[cosine_similarity(vec1, vec2) for vec2 in vectors] for vec1 in vectors])


def plot(x, title=''):
  plt.figure(figsize=(7,4))
  plt.pcolormesh(x,
                # Use a diverging colormap so it's clear where zero is.
                cmap='RdBu', vmin=-1, vmax=1)
  plt.xlim([0,d_model])
  plt.ylabel('num_positions')
  plt.xlabel('d_model')
  plt.title(title)

  plt.colorbar()

def plot_similarity(x, title):
  similarity_matrix = cosine_similarity_matrix(x)
  plt.figure(figsize=(5,4))
  plt.pcolormesh(similarity_matrix,
                # Use a diverging colormap so it's clear where zero is.
                cmap='RdBu', vmin=-1, vmax=1)
  plt.xlim([0,num_positions])
  plt.ylabel(num_positions)
  plt.xlabel(num_positions)
  plt.title(f'plot_similarity {title}')
  plt.colorbar()

def make_circle():
  x = torch.arange(0, d_model)
  y = torch.arange(0, num_positions)
  arr = torch.zeros((y.shape[0], x.shape[0]))
  cx = d_model / 2.
  cy = num_positions / 2.
  r = num_positions / 2.
  mask = (x[None,:]-cx)**2 + (y[:,None]-cy)**2 < r**2
  arr[mask] = 1.
  return arr


In [13]:

# origin_x = 2 * np.random.rand(num_positions, d_model) - 1
origin_x = torch.rand(num_positions, d_model) - torch.rand(num_positions, d_model)
# origin_x = torch.ones((num_positions, d_model))
# origin_x = make_circle()
origin_x = origin_x[None, :]

print(origin_x, origin_x.shape)

tensor([[[ 0.1704, -0.1164, -0.1910,  ...,  0.0407,  0.1671,  0.3889],
         [ 0.3169,  0.4143,  0.6562,  ..., -0.5193,  0.1040,  0.1081],
         [ 0.5557,  0.2067,  0.2081,  ..., -0.3448,  0.3887,  0.6172],
         ...,
         [ 0.6988,  0.2822,  0.7853,  ...,  0.1062, -0.0400,  0.6083],
         [-0.7178, -0.7389, -0.6050,  ..., -0.1638,  0.1126, -0.0394],
         [-0.0336, -0.3020,  0.1524,  ..., -0.0475, -0.7326,  0.3057]]]) torch.Size([1, 500, 512])


1.  position ecoding



\begin{align*}
    PE_{(pos,2i)} = sin(pos / 10000^{2i/dmodel}) \\
    PE_{(pos,2i+1)} = cos(pos / 10000^{2i/dmodel})
\end{align*}

In [14]:
i_s = torch.arange(d_model//2)
# print(i_s)
min_rate = 1/10000
exponents = (2 / d_model) * i_s # [0, 1]
# print(exponents)
omegas = min_rate**exponents # [1, 0.0001]
omegas = omegas[None, :]
# print(omegas, omegas.shape)


In [15]:
positions = torch.arange(num_positions)
positions = positions[:, None]
# print(positions, positions.shape)
# print((positions/omegas))
sines = torch.sin(positions*omegas)
cosines = torch.cos(positions*omegas)
# print(sines)
# print(cosines)
pos_encoding = torch.concatenate([sines, cosines], axis=-1)
pos_encoding_ziped = torch.reshape(torch.stack((sines, cosines), axis=-1), (num_positions, d_model))



# print(pos_encoding, pos_encoding.shape)

In [16]:
xs_positioned = pos_encoding + origin_x
xs_positioned_ziped = pos_encoding_ziped + origin_x


In [17]:
print('origin_x norm', torch.linalg.norm(origin_x,axis=2))
print('origin_x xs_positioned', torch.linalg.norm(xs_positioned,axis=2))

plot(origin_x[0], 'origin_x')
plot(pos_encoding, 'pos_encoding')
plot(pos_encoding_ziped, 'pos_encoding_ziped')
plot(xs_positioned[0], 'xs_positioned')
plot(xs_positioned_ziped[0], 'xs_positioned_ziped')
plot_similarity(origin_x[0], 'origin_x')
plot_similarity(pos_encoding, 'pos_encoding')
plot_similarity(xs_positioned[0], 'xs_positioned')
plot_similarity(xs_positioned_ziped[0], 'xs_positioned_ziped')




Output hidden; open in https://colab.research.google.com to view.

2. RoPE

https://spaces.ac.cn/archives/8265

In [18]:
thetas = omegas
# print(thetas, thetas.shape)

theta_is =  torch.stack((thetas, thetas), axis=-1)
theta_is = torch.reshape(theta_is, (1, d_model))

# print(theta_is, theta_is.shape)


In [19]:
def neg_swaped_x(x):
  print(x.shape)
  x = x.transpose(1, 2)

  triangle = torch.arange(d_model)
  x_inter_nagtive = torch.where(triangle % 2 == 0, -1, 1)
  # print(x_inter_nagtive, x_inter_nagtive.shape)

  rs = torch.arange(d_model)
  filter_0 = torch.where(rs % 2 == 0, True, False)
  filter_1 = torch.where(rs % 2 == 1, True, False)
  # print(filter_0)
  # print(filter_1)
  x0 = x[filter_0[None, :]]
  x1 = x[filter_1[None, :]]

  x_new =  torch.stack((x1, x0), axis=-1)
  # need transpose !
  x_new = x_new.transpose(0, 1)
  x_new = torch.reshape(x_new, (num_positions, d_model))
  x_new = x_new * x_inter_nagtive
  # print(x_new)
  return x_new



In [20]:

# print(positions, positions.shape)
# print(theta_is, theta_is.shape)

pos_theta = positions * theta_is
# print(pos_theta, pos_theta.shape)


x_1 = origin_x * torch.cos(pos_theta)
x_2 = neg_swaped_x(origin_x) * torch.sin(pos_theta)
x_roped = x_1 + x_2

plot(torch.cos(pos_theta), 'torch.cos(pos_theta)')
plot(torch.sin(pos_theta), 'torch.sin(pos_theta)')

print('origin_x norm', torch.linalg.norm(origin_x,axis=2))
print('origin_x x_roped', torch.linalg.norm(x_roped,axis=2))

plot(origin_x[0], 'origin_x')
plot(x_roped[0], 'x_roped')
plot_similarity(origin_x[0], 'origin_x')
plot_similarity(x_roped[0], 'x_roped')







Output hidden; open in https://colab.research.google.com to view.



* Changes the Norm length?        Sinusoidal 2x; RoPE 1x;
* Changes visually?            Sinusoidal YES; RoPE YES and NO(for pure random nosie)
* Change the similarity of adjacent? Sinusoidal increase; RoPE decrease
